In [19]:
import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf
import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from math import radians, cos, sin, asin, sqrt
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

from math import radians, cos, sin, asin, sqrt

import copy
import sys

import geopandas as gpd
import numpy as np

import gpflow


code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel, PerturbedBPRMLPModel
from make_datasets import make_data


from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood
gpflow.config.default_float()


from zinf_gp.metrics import normcdf, fixed_top_X

from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from functools import partial


In [2]:
run_template = '{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}_{lr}'
#/annual_group_normal_2000_-auto_200_0.01

In [3]:
time = 'annual'
loc = 'tract'
model = 'normal'
start_year = '2010'
cov = '-auto'
num_inducing = '400'
test_years=2
data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/new_big_run'

In [20]:
town_map = pd.read_csv(os.path.join(data_dir, 'town_tract_map.csv'), dtype=str)
group_map = gpd.read_file(os.path.join(data_dir, 'tract_group_map'), dtype=str)

# test y always comes from quarterly tract
y_timesteps_per_year = 1
file_name = f'clean_annual_tract'
data_path = os.path.join(data_dir, file_name)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile',
              'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat', 'lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)

last_train_year = 2018

test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year + test_years)][y_idx_cols]
starting_y_timestep = int(test_y[timestep_col].min())

sorted_y_timesteps = test_y[timestep_col].unique()
sorted_y_timesteps.sort()

x_timesteps_per_year = {'quarter': 4, 'semi': 2, 'annual': 1}[time]

file_name = f'clean_{time}_{loc}'
data_path = os.path.join(data_dir, file_name)

#data_gdf = gpd.read_file(data_path)

test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year + test_years)][x_idx_cols]

starting_x_timestep = int(test_x[timestep_col].min())

test_timesteps_per_year = max(y_timesteps_per_year, x_timesteps_per_year)
test_timesteps = test_timesteps_per_year * test_years

x_repeats = int(test_timesteps_per_year / x_timesteps_per_year)
y_repeats = int(test_timesteps_per_year / y_timesteps_per_year)

sorted_x_timesteps = test_x[timestep_col].unique()
sorted_x_timesteps.sort()

x_timesteps = [timestep for timestep in sorted_x_timesteps for _ in range(x_repeats)]

y_timesteps = [timestep for timestep in sorted_y_timesteps for _ in range(y_repeats)]


best_model = 'n0.3_samp50_lr0.01_hs50_10_lb5'
noise = 0.3
perturbation_samples = 50
hidden_sizes = [50, 10]
lookback_years=5
learning_rate=0.01
add_spacetime=True
add_svi=True
epochs = 3000
seed = 360
time_window = lookback_years
timesteps_per_year = 1
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020


multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=timesteps_per_year)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=timesteps_per_year)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=timesteps_per_year)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)

In [6]:
best_elbo = -np.inf
for lr in ['0.01','0.001']:
    for model in ['normal', 'poisson']:
        for start_year in ['2000','2010']:
            for cov in ['-auto','all']:
                for num_inducing in ['200','400']:

                    this_run = run_template.format(time=time, loc=loc,
                                                   model=model, start_year=start_year,
                                                   cov=cov,
                                                   num_inducing=num_inducing, lr=lr)
                    try:
                        with open(os.path.join(log_dir, this_run, 'model.mod'), 'rb') as f:
                            predictor = pickle.load(f)
                        with open(os.path.join(log_dir, this_run, 'stats.csv'), 'rb') as f:
                            stats = pd.read_csv(f)
                            elbo = stats.iloc[-1, :][['elbo']].values[0]
                    except(FileNotFoundError):
                        print(os.path.join(log_dir,this_run))
                        continue

                    if elbo > best_elbo:
                        best_elbo = elbo
                        print(this_run)
                        best_predictor = copy.deepcopy(predictor)

2023-07-06 06:41:47.048612: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-06 06:41:47.048667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2023-07-06 06:41:47.048677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: s1cmp008.pax.tufts.edu
2023-07-06 06:41:47.048843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 515.65.1
2023-07-06 06:41:47.048879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.65.1
2023-07-06 06:41:47.048886: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 515.65.1


annual_tract_normal_2000_-auto_200_0.01
annual_tract_normal_2000_-auto_400_0.01
annual_tract_normal_2010_-auto_200_0.01
annual_tract_normal_2010_-auto_400_0.01
annual_tract_poisson_2000_-auto_200_0.01
annual_tract_poisson_2000_-auto_400_0.01
annual_tract_poisson_2010_-auto_200_0.01
annual_tract_poisson_2010_-auto_400_0.01


In [7]:
xtops = []
for year in range(test_years):
    xtop_year = []
    max_timesteps = max(x_timesteps_per_year, y_timesteps_per_year)
    for x_time, y_time in zip(x_timesteps[year * max_timesteps:(year + 1) * max_timesteps],
                              y_timesteps[year * max_timesteps:(year + 1) * max_timesteps]):
        test_x_time = test_x[test_x[timestep_col] == x_time]
        test_y_time = test_y[test_y[timestep_col] == y_time]
        _, _, _, fmean, fvar, gmean, gvar, _, _ = best_predictor.build_predict(test_x_time.loc[:, features_only].values)
        g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
        pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])


        y_index = test_y_time[geography_col]
        sampled_size = int(0.95*len(y_index))
        assert(sampled_size==1539)
        sampled_xtops = []
        for _ in range(100):
            sampled_index = np.random.choice(y_index, sampled_size,replace=False)
            sampled_test = test_y_time.set_index(geography_col).loc[sampled_index, outcome_col]
            sampled_pred = pred_df.loc[sampled_index]
            result = fixed_top_X(sampled_test, sampled_pred, 100)[-1]
            sampled_xtops.append(result)

        xtop_ptiles = np.percentile(sampled_xtops, [2.5, 50, 97.5])
        xtop_year.append(xtop_ptiles)
    annual_avg = np.array(xtop_year).mean(axis=0)
    xtops.append(annual_avg)

KeyboardInterrupt: 

In [9]:
max_timesteps

1

In [10]:
xtops

[array([0.44046947, 0.45405563, 0.47257506]),
 array([0.43850162, 0.45537507, 0.48084863])]

In [10]:
xtops = []
maes = []
rmses = []
preds = []
for year in range(test_years):
    xtop_year = []
    max_timesteps = max(x_timesteps_per_year, y_timesteps_per_year)
    for x_time, y_time in zip(x_timesteps[year * max_timesteps:(year + 1) * max_timesteps],
                              y_timesteps[year * max_timesteps:(year + 1) * max_timesteps]):
        test_x_time = test_x[test_x[timestep_col] == x_time]
        test_y_time = test_y[test_y[timestep_col] == y_time]
        _, _, _, fmean, fvar, gmean, gvar, _, _ = best_predictor.build_predict(test_x_time.loc[:, features_only].values)
        g_cond = tf.math.softplus(fmean * normcdf(gmean)).numpy()
        pred_df = pd.Series(g_cond.squeeze(), index=test_x_time[geography_col])
        preds.append(pred_df)

        result = fixed_top_X(test_y_time.set_index(geography_col).loc[:, outcome_col], pred_df, 100)[-1]
        
        rmses.append(sqrt(mean_squared_error(test_y_time.set_index(geography_col).loc[:, outcome_col], pred_df)))
        
        maes.append(mean_absolute_error(test_y_time.set_index(geography_col).loc[:, outcome_col], pred_df))

        xtop_year.append(result)
    annual_avg = np.array(xtop_year).mean(axis=0)
    xtops.append(annual_avg)

In [22]:

this_2019 = multiindexed_gdf.loc[idx[:,19],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
this_2020 = multiindexed_gdf.loc[idx[:,20],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']

In [26]:
%%time
bprs_2019=[]
bprs_2020=[]
rng = np.random.default_rng(seed=360)
removed_tracts = 250
num_tracts = test_x_BSF_flat.shape[1]
num_sampled = num_tracts - removed_tracts
K=100

for _ in range(100):
    sampled_indicies = rng.choice(range(num_tracts), size=num_sampled, replace=False)
    
    bprs_2019.append(fixed_top_X_new(this_2019[sampled_indicies],
                            preds[0][sampled_indicies],
                            X=K)[-1])
    bprs_2020.append(fixed_top_X_new(this_2020[sampled_indicies],
                            preds[1][sampled_indicies],
                            X=K)[-1])

CPU times: user 60 s, sys: 176 ms, total: 1min
Wall time: 59.9 s


In [27]:
avg_bprs = (np.array(bprs_2019)+np.array(bprs_2020))/2
np.percentile(avg_bprs,2.5), np.mean(avg_bprs), np.percentile(avg_bprs, 97.5)

(0.43938636480679005, 0.46125555906681526, 0.4868839017524691)

In [28]:
xtops

[0.4444444444444446, 0.45290581162324633]

In [29]:
np.mean(xtops)

0.44867512803384546

In [19]:
xtops

[0.4444444444444446, 0.45290581162324633]

In [20]:
np.mean(xtops)

0.44867512803384546

In [21]:
np.mean(maes)

1.1334389263152322

In [22]:
np.mean(rmses)

3.4415631203967694

In [9]:
preds

[geoid
 25001010100    0.755169
 25001010206    0.755858
 25001010208    0.754911
 25001010304    0.755814
 25001010306    0.756159
                  ...   
 25027761100    0.755977
 25027761200    0.756139
 25027761300    0.756042
 25027761401    0.755985
 25027761402    0.756044
 Length: 1620, dtype: float64,
 geoid
 25001010100    0.796505
 25001010206    0.797219
 25001010208    0.796238
 25001010304    0.797173
 25001010306    0.797530
                  ...   
 25027761100    0.797342
 25027761200    0.797510
 25027761300    0.797410
 25027761401    0.797350
 25027761402    0.797412
 Length: 1620, dtype: float64]

In [11]:
def fixed_top_X_new(true_qtr_val, pred_qtr_val, X=10):
    top_X_predicted = pred_qtr_val.sort_values(ascending=False).iloc[:X]
    top_X_true = true_qtr_val.sort_values(ascending=False).iloc[:X]

    undisputed_top_predicted = top_X_predicted[top_X_predicted > top_X_predicted.min()]
    num_tied_spots = X - len(undisputed_top_predicted)
    undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
    num_true_ties = X - len(undisputed_top_true)

    tied_top_predicted = pred_qtr_val[pred_qtr_val == top_X_predicted.min()]
    tied_top_true = true_qtr_val[true_qtr_val == top_X_true.min()]

    error_in_top_true_ties = np.abs(tied_top_true - pred_qtr_val[tied_top_true.index]).sort_values(ascending=True)
    error_in_top_pred_ties = np.abs(true_qtr_val[tied_top_predicted.index] - tied_top_predicted).sort_values(
        ascending=True)
    top_true_tied_geoids = error_in_top_true_ties.iloc[:num_true_ties].index
    top_pred_tied_geoids = error_in_top_pred_ties.iloc[:num_tied_spots].index

    best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
    best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

    # True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
    best_possible_true = true_qtr_val[best_possible_top_true_geoids]
    best_possible_pred = true_qtr_val[best_possible_top_pred_geoids]
    assert (len(best_possible_true) == X)
    assert (len(best_possible_pred) == X)

    best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
    best_possible_ratio = np.abs(best_possible_pred).sum() / np.abs(best_possible_true).sum()

    bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
    this_idx = undisputed_top_predicted.index.values
    bootstrapped_all_indices = [np.concatenate((this_idx,bootstrap_index)) for bootstrap_index in bootstrapped_tied_indices]
    #bootstrapped_all_indices = [pd.Index.union(undisputed_top_predicted.index,
    #                                           bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]
    
    denominator =  top_X_true.sum()
    numerators = [true_qtr_val[indicies].sum() for indicies in bootstrapped_all_indices] 

    bootstrapped_absolute = np.mean([np.abs(denominator - numerator)
                                     for numerator in numerators])
    bootstrapped_ratio = np.mean([numerator / denominator
                                  for numerator in numerators])

    return best_possible_absolute, best_possible_ratio, bootstrapped_absolute, bootstrapped_ratio

In [25]:
preds[0]

geoid
25001010100    0.755169
25001010206    0.755858
25001010208    0.754911
25001010304    0.755814
25001010306    0.756159
                 ...   
25027761100    0.755977
25027761200    0.756139
25027761300    0.756042
25027761401    0.755985
25027761402    0.756044
Length: 1620, dtype: float64